In [1]:
import numpy as np

from qiskit.circuit import QuantumCircuit, Parameter
from qiskit.quantum_info import SparsePauliOp, Statevector

from surfer.gradient import ReverseGradient
from surfer.qfi import ReverseQFI

In [23]:
t = 1
op = SparsePauliOp(["I", "Z"], coeffs=[0, t])

In [24]:
x, y, z = [Parameter(s) for s in "xyz"]
circuit = QuantumCircuit(1)
# circuit.h(0)
circuit.rz(x, 0)
circuit.ry(y, 0)
circuit.rz(z, 0)

In [25]:
initial_parameters = [0, np.pi/2, 0]
final_time = 1
timestep = 0.01

qfi = ReverseQFI(do_checks=False)
gradient = ReverseGradient(do_checks=False, partial_gradient=True)

In [26]:
parameters = [initial_parameters]
time = 0
while time <= final_time:
    op.coeffs[1] = time
    energy = Statevector(circuit.bind_parameters(parameters[-1])).expectation_value(op)
    
    qgt = qfi.compute(circuit, parameters[-1]) / 4
    
    shifted = op - SparsePauliOp(["I"], coeffs=[energy])
    b = -np.imag(gradient.compute(shifted, circuit, parameters[-1]))
        
    derivative = np.linalg.solve(qgt + 0.001 * np.eye(len(initial_parameters)), b)
    parameters.append(parameters[-1] + timestep * derivative)
    time += timestep

In [27]:
state = Statevector(circuit.bind_parameters(parameters[-1]))
state

Statevector([0.62289306-0.33467034j, 0.62289306+0.33467034j],
            dims=(2,))


In [34]:
evolved = 1 / np.sqrt(2) * np.array([
    np.exp(-1j / 2 * final_time ** 2), 
    np.exp(1j / 2 * final_time ** 2)
])

In [35]:
evolved

array([0.62054458-0.33900505j, 0.62054458+0.33900505j])

In [41]:
state.equiv(evolved, atol=1e-2)

True

In [10]:
evolved[0] / state.data[0], evolved[1] / state.data[1]

((0.874218650927579-0.48553244008033425j),
 (0.8742186509275655+0.4855324400803594j))

### Static example

In [16]:
op = SparsePauliOp(["Z"], coeffs=[1])

In [17]:
initial_parameters = [0, np.pi/2, 0]
final_time = 1
timestep = 0.01

qfi = ReverseQFI(do_checks=False)
gradient = ReverseGradient(do_checks=False, partial_gradient=True)

In [18]:
parameters = [initial_parameters]
time = 0
while time <= final_time:
    energy = Statevector(circuit.bind_parameters(parameters[-1])).expectation_value(op)
    
    qgt = qfi.compute(circuit, parameters[-1]) / 4
    
    shifted = op - SparsePauliOp(["I"], coeffs=[energy])
    b = -np.imag(gradient.compute(shifted, circuit, parameters[-1]))
        
    derivative = np.linalg.solve(qgt + 0.001 * np.eye(len(initial_parameters)), b)
    parameters.append(parameters[-1] + timestep * derivative)
    time += timestep

In [20]:
state = Statevector(circuit.bind_parameters(parameters[-1]))
state

Statevector([0.38441894-0.593483j, 0.38441894+0.593483j],
            dims=(2,))


In [21]:
expected = 1 / np.sqrt(2) * np.array([
    np.exp(-1j * final_time),
    np.exp(1j * final_time)
])

In [22]:
expected

array([0.38205142-0.59500984j, 0.38205142+0.59500984j])